In [1]:
import sys
import re
import datetime
import unicodedata
import csv
import logging
import argparse

# I kind of hate this
#from __future__ import print_function

import httplib2
import os

from apiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage 
from apiclient import errors 

# SCOPES = 'https://www.googleapis.com/auth/drive'
#SCOPES = 'https://www.googleapis.com/auth/drive.file' # try this one!
# SCOPES = 'https://www.googleapis.com/auth/drive.metadata.readonly'
CLIENT_SECRET_FILE = 'client_secret_qs.json'
# APPLICATION_NAME = 'Drive API Python Quickstart'

In [2]:
def strip_accents(s):
    s = unicode(s, "utf-8")
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


In [3]:
def get_valid_filename(s):
    """
    Return the given string converted to a string that can be used for a clean
    filename. Remove leading and trailing spaces
    """
    s = s.strip()
    s = strip_accents(s)
    return re.sub(r'(?u)[^-\w., ]', '', s)

In [4]:
def fileRead(fname):
    '''
    read a file into a list, strip out all accented and special characters, leading spaces
    '''
    lines = []
    try:
        with open(fname) as f:
            for each in f:
                each = get_valid_filename(each)
                lines.append(each.strip('\n'))
            return(lines)
    except (OSError, IOError) as e:
        print 'error reading file:', fname, e
        return(False)

In [5]:
def pathify(parts = [], basepath = ''):
    '''
    create a path from a list of strings
    
    accepts:
        * parts (list): list of strings ['part_one', 'part_two', 'part_three']
        * basepath (string): string to append to the start of the path (default: ./)
    
    returns:
        * path: basePath/part_one/part_two/part_three/
    '''
    path = basepath
    if len(parts ) > 0:
        for each in parts:
            if re.match('.*\/$', each):
                path = path + each
            else:
                path = path + each + '/'
    return(path)

In [6]:
def getCredentials(client_secret = ''):
    SCOPES = 'https://www.googleapis.com/auth/drive' 
    home_dir = os.path.expanduser('~')
    credential_dir = os.path.join(home_dir, '.credentials')
#     flags = argparser.parse_args('--auth_host_name localhost --logging_level INFO'.split())
    flags = tools.argparser.parse_args([])
    if not os.path.exists(credential_dir):
        try:
            os.makedirs(credential_dir)
        except (IOError, OSError) as e:
            print(e)
        

    credential_path = os.path.join(credential_dir, 'drive-python.json')

    store = Storage(credential_path)
    credentials = store.get()
    if not credentials or credentials.invalid:
        flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
        flow.user_agent = 'foo'
#         credentials = tools.run_flow(flow, store, None)
        credentials = tools.run_flow(flow, store, flags)
    return(credentials)

In [ ]:
def XgetCredentials(client_secret = ''):
    home_dir = os.path.expanduser('~')
    credential_dir = os.path.join(home_dir, '.credentials')
#     flags = argparser.parse_args('--auth_host_name localhost --logging_level INFO'.split())
    flags = tools.argparser.parse_args([])
    if not os.path.exists(credential_dir):
        try:
            os.makedirs(credential_dir)
        except (IOError, OSError) as e:
            print(e)
        

    credential_path = os.path.join(credential_dir, 'drive-python.json')

    store = Storage(credential_path)
    credentials = store.get()
    if not credentials or credentials.invalid:
        flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
        flow.user_agent = 'foo'
#         credentials = tools.run_flow(flow, store, None)
        credentials = tools.run_flow(flow, store, flags)
    return(credentials)

In [7]:
class GDriveError(Exception):
    pass

In [8]:
class gDrive():
    '''
    creates a google drive interface object
    
    Accepts:
    google drive v3 service object: (discover.build('drive', 'v3', credentials = credentials_object)
    
    '''
    def __init__(self, object):
        if  not isinstance(object, discovery.Resource):
            print ('Error: googleapicleint.discovery.Resource object expected')
            print ('{:>5}create a resource object:'.format(''))
            print ('{:>10}credentials = getCredentials(credJSON = "cleint_secret.json")'.format(''))
            print ('{:>10}service = discovery.build("drive", "v3", credentials=credentials)'.format(''))
            print ('{:>10}myDrive = gDrive(service)'.format(''))
            return(None)
        self.service = object
        # https://developers.google.com/drive/v3/web/mime-types
        self.mimeTypes = {'audio': 'application/vnd.google-apps.audio',
                          'docs': 'application/vnd.google-apps.document',
                          'drawing': 'application/vnd.google-apps.drawing',
                          'file': 'application/vnd.google-apps.file',
                          'folder': 'application/vnd.google-apps.folder',
                          'forms': 'application/vnd.google-apps.form',
                          'mymaps': 'application/vnd.google-apps.map',
                          'photos': 'application/vnd.google-apps.photo',
                          'slides': 'application/vnd.google-apps.presentation',
                          'scripts': 'application/vnd.google-apps.script',
                          'sites': 'application/vnd.google-apps.sites',
                          'sheets': 'application/vnd.google-apps.spreadsheet',
                          'video': 'application/vnd.google-apps.video'}
        
        # fields to include in partial responses
        # https://developers.google.com/apis-explorer/#p/drive/v3/drive.files.create
        self.fields = ['id', 'parents', 'mimeType', 'webViewLink', 'size', 'createdTime', 'trashed', 'kind', 'name']
    
#     types = property()
    
    @property
    def types(self):
        '''
        Display supported mimeTypes
        '''
        print('supported mime types:')
        for key in self.mimeTypes:
            #print('%10s: %s' % (key, self.mimeTypes[key]))
            print('{:8} {val}'.format(key+':', val=self.mimeTypes[key]))
    
#     def quote(self, string):
#         '''
#         add double quotes arounda string
#         '''
#         return('"'+str(string)+'"')
    

    
    def add(self, name = None, mimeType = False, parents = None, 
            fields = 'webViewLink, mimeType, id', sanitize = True):
        '''
        add a file to google drive:
        
        args:
            name (string): human readable name
            mimeType (string): mimeType (see self.mimeTypes for a complete list)
            parents (list): list of parent folders
            fields (comma separated string): properties to query and return any of the following:
                'parents', 'mimeType', 'webViewLink', 
                'size', 'createdTime', 'trashed'
                'id'
            sanitize (bool): remove any field options that are not in the above list - false to allow anything
            
        '''

        fieldsExpected = self.fields
        fieldsProcessed = []
        fieldsUnknown = []
        
        if sanitize:
            # remove whitespace and unknown options
            for each in fields.replace(' ','').split(','):
                if each in fieldsExpected:
                    fieldsProcessed.append(each)
                else:
                    fieldsUnknown.append(each)
        else:
            fieldsProcessed = fields.split(',')
            
        if len(fieldsUnknown) > 0:
            logging.warn('unrecognized fields: {}'.format(fieldsUnknown))
        
        
        body={}
        if name is None:
            logging.error('expected a folder or file name')
            return(False)
        else:
            body['name'] = name
        
        if mimeType in self.mimeTypes:
            body['mimeType'] = self.mimeTypes[mimeType]
        
        if isinstance(parents, list):
            body['parents'] = parents
        elif parents:
            body['parents'] = [parents]
        
        apiString = 'body={}, fields={}'.format(body, ','.join(fieldsProcessed))
        logging.debug('api call: files().create({})'.format(apiString))
        try:
            result = self.service.files().create(body=body, fields=','.join(fieldsProcessed)).execute()
            return(result)
        except errors.HttpError as e:
            raise GDriveError(e)
            return(False)
        
        
        #body = {'name':'release the schmoo!', 'mimeType':'application/vnd.google-apps.folder', 'parents':["0BzC-V2QIsGRGWXNxNmhjc0FITDQ"]}
# service.files().create(body=body).execute()
        

    
    def search(self, name = None, trashed = None, mimeType = False, fuzzy = False, date = None, dopperator = '>', 
               parents = None, orderBy = 'createdTime', quiet = True):
        '''
        search for an item by name and other properties in google drive
        
        args:
            name (string): item name in google drive - required
            trashed (bool): item is not in trash - default None (not used)
            mimeType = (string): item is one of the known mime types (gdrive.mimeTypes) - default None
            fuzzy = (bool): substring search of names in drive
            date = (RFC3339 date string): modification time date string (YYYY-MM-DD)
            dopperator (date comparison opprator string): <, >, =, >=, <=  - default >
            parents = (string): google drive file id string
            orderBy = (comma separated string): order results assending by keys below - default createdTime:
                        'createdTime', 'folder', 'modifiedByMeTime', 
                        'modifiedTime', 'name', 'quotaBytesUsed', 
                        'recency', 'sharedWithMeTime', 'starred', 
                        'viewedByMeTime'
            fields (comma separated string): properties to query and return any of the following:
                'parents', 'mimeType', 'webViewLink', 
                'size', 'createdTime', 'trashed'
                'id'
            sanitize (bool): remove any field options that are not in the above list - false to allow anything
                        
                        
            
        returns:
            list of file dict
        '''
        features = ['name', 'trashed', 'mimeType', 'date', 'parents']
        build = {'name' : 'name {} "{}"'.format(('contains' if fuzzy else '='), name),
                 'trashed' : 'trashed={}'. format(trashed),
                 'mimeType' : 'mimeType="{}"'.format(self.mimeTypes[mimeType] if mimeType in self.mimeTypes else ''),
                 'parents': '"{}" in parents'.format(parents),
                 'date': 'modifiedTime{}"{}"'.format(dopperator, date)}

        
        # provides for setting trashed to True/False if the input is not None
        if not isinstance(trashed, type(None)):
            # set to true as the variable is now in use, but it's value has been set above
            trashed = True
        
        qList = []

        # evaluate feature options; if they are != None/False, use them in building query
        for each in features:
            if eval(each):
                qList.append(build[each])
                
        if not quiet:
            print(' and '.join(qList))
        
        apiString = 'q={}, orderBy={})'.format(' and '.join(qList), orderBy)
        logging.debug('apicall: files().list({})'.format(apiString))
        try:
            # build a query with "and" statements
            result = self.service.files().list(q=' and '.join(qList), orderBy=orderBy).execute()
            return(result)
        except errors.HttpError as e:
            raise GDriveError(e)
            return(False)

    def ls(self, *args, **kwargs):
        '''
        List files in google drive using any of the following properties:
            
        accepts:
            name (string): item name in google drive - required
            trashed (bool): item is not in trash - default None (not used)
            mimeType = (string): item is one of the known mime types (gdrive.mimeTypes) - default None
            fuzzy = (bool): substring search of names in drive
            date = (RFC3339 date string): modification time date string (YYYY-MM-DD)
            dopperator (date comparison opprator string): <, >, =, >=, <=  - default >
            parent = (string): google drive file id string    
        '''
        try:
            result = self.search(*args, **kwargs)
            for eachFile in result.get('files', []):
                print('name: {f[name]}, ID:{f[id]}, mimeType:{f[mimeType]}'.format(f=eachFile))
            return(result)
        except GDriveError as e:
            raise GDriveError(e)
            
    
    
    def getprops(self, fileId = None, fields = 'parents, mimeType, webViewLink', sanitize=True):
        '''
        get a file or folder's properties based on google drive fileId
        
        for a more complete list: https://developers.google.com/drive/v3/web/migration
        
        args:
            fileId (string): google drive file ID
            fields (comma separated string): properties to query and return any of the following:
                'parents', 'mimeType', 'webViewLink', 'size', 'createdTime', 'trashed'
            sanitize (bool): remove any field options that are not in the above list - false to allow anything
            
        returns:
            list of dictionary - google drive file properties
            
        raises GDriveError
        '''
        fieldsExpected = self.fields
        
        fieldsProcessed = []
        fieldsUnknown = []

        if sanitize:
            # remove whitespace and unknown options
            for each in fields.replace(' ','').split(','):
                if each in fieldsExpected:
                    fieldsProcessed.append(each)
                else:
                    fieldsUnknown.append(each)
        else:
            fieldsProcessed = fields.split(',')
        if len(fieldsUnknown) > 0:
            print ('unrecognized fields: {}'.format(fieldsUnknown))
        
        apiString = 'fileId={}, fields={}'.format(fileId, ','.join(fieldsProcessed))
        logging.debug('files().get({})'.format(apiString))
        try:
            result = self.service.files().get(fileId=fileId, fields=','.join(fieldsProcessed)).execute()
            return(result)
        except errors.HttpError as e:
            raise GDriveError(e)
            return(False)
        

    def parents(self, fileId):
        """get a file's parents.

        Args:
            fileId: ID of the file to print parents for.
        
        raises GDriveError
        """
        apiString = 'fileId={}, fields="parents"'.format(fileId)
        logging.debug('api call: {}'.format(apiString))
        try:
            parents = self.service.files().get(fileId=fileId, fields='parents').execute()
            return(parents)
        except errors.HttpError as e:
            raise GDriveError(e)
            return(False)
    
    def rm(self):
        pass

In [9]:
# alternatively pass in the configuration object?
def gDrivePopulate(gdBaseFolder = '', folderList = '', client_secret = ''):
    
    BASEFOLDER = '0B9WTleJ1MzaYcmdmTWNNNF9pa1E'
    
    INPUTFILE = './student_export.text'

    # this needs to be moved out of this def and moved into the main loop of the other program
    # init the log; this removes any old log handlers (this is particularly useful when testing in an IDE)
    log = logging.getLogger()
    logging.getLogger("googleapiclient").setLevel(logging.WARNING)

    if len(log.handlers) > 0:
        for each in range(0, len(log.handlers)):
            log.removeHandler(log.handlers[0])
            
    # set the log format:
    # [  DEBUG 2017-02-12 19:14] loading module: requests
    logFormatter = logging.Formatter('[%(levelname)8s %(asctime)s] %(message)s', '%Y-%m-%d %H:%M')
    #consoleFormatter = logging.Formatter('[%(levelname)-8s] %(message)s')
    consoleFormatter = logging.Formatter('[%(levelname)-8s - %(name)s] %(message)s')
    # set root logger
    rootLogger = logging.getLogger()       
    
    # silence INFO from the google api client
    logging.getLogger('discovery').setLevel(logging.ERROR)

    
    # add a conshole handle to the root logger
    consoleHandler = logging.StreamHandler(sys.stdout)
#     consoleHandler.setFormatter(logFormatter)
    consoleHandler.setFormatter(consoleFormatter)
    rootLogger.addHandler(consoleHandler) 
    
    rootLogger.setLevel(logging.DEBUG)
    
    
    logging.info('checking google credentials')
    credentials = getCredentials(client_secret = 'client_secret_qs.json')
    http = credentials.authorize(httplib2.Http())
    try:
#         service = discovery.build('drive', 'v3', credentials=credentials)
        service = discovery.build('drive', 'v3', http=http, cache_discovery=False)
    except Exception as e:
        logging.critical('Error communicating with Google: {}'.format(e))
        logging.critical('exiting')
        return(False)

    myDrive = gDrive(service)

    # path to desktop of this user (this should work on macs)
    desktopPath = pathify([os.path.expanduser('~')], 'Desktop')

    # Fields expected in CSV file containing student data
    # move this into a resource file? - see the folder list below
    expected = ['ClassOf', 'LastFirst', 'Student_Number']

    # map of CSV fields to elements in list
    headerMap = {}

    # list of folders to populate into each students' folder
    if os.path.exists(desktopPath+'folders.txt'):
        fileList = desktopPath + 'folders.txt'
    else:
        fileList = './folders.txt'

    # list of student information
    studentLinks = []
    

    # list containing CSV
    studentCSV = []

    # list of folders to add under each students' path
    logging.info('reading list of folders to create for each student:')
    gradeFolders = fileRead(fileList)

#         print('input file: {}'.format(sys.argv[1]))
#         studentInfo = sys.argv[1]

    studentInfo = INPUTFILE

    if gradeFolders:
        logging.info('Folders to be created for each student: {}'.format(len(gradeFolders)))
        logging.info('{:>5}List of folders to be created for each student:'.format(''))
        
        for folder in gradeFolders:
            logging.info('{:>10}{}'.format('', folder))

    else:
        logging.critical('default grade folders file is missing')
        logging.critical('please place a file named "folders.txt" containing a list of folders (one on each line) on the Desktop')
        logging.critical('exiting')
        return(False)

    # read the csv file in Universal newline mode (rU)
    logging.info('reading student CSV file')
    try:
        with open(studentInfo, 'rU') as csvfile:
            csvreader = csv.reader(csvfile)
            for row in csvreader:
                studentCSV.append(row)
    except (OSError, IOError) as e:
        logging.critical('error reading file: {}\n{}',format(studentInfo, e))
        logging.critical('exiting')
        return(False)

    # check if there is content in the CSV
    if len(studentCSV) > 1:
        logging.info('{} student records found in {}'.format(len(studentCSV)-1, studentInfo))
#         print('{} student records found in {}'.format(len(studentCSV)-1, studentInfo))
    else:
        logging.critical('no student records found in {}'.format(studentInfo))
        logging.critical('exiting') 
        return(False)

    # check for expected headers in CSV data
    # pop the headers from the list
    csvHeader = studentCSV.pop(0)
    missingHeaders = []
    for each in expected:
        if each not in csvHeader:
            missingHeaders.append(each)

    if len(missingHeaders) > 0:
        loggig.critical('error reading student student CSV fle: {}'.format(studentInfo))
        for each in missingHeaders:
            print('{:>5} missing header: {}'.format(studentInfo, each))
        logging.critical('please recreate file {} with all required headers:'.format(studentInfo))
        logging.critical('{:>5}'.format(expected))
        logging.critical('stopping')
        return(False)

    # map headers to their index
    for index, value in enumerate(csvHeader):
        headerMap[value]=index
    
    # process the CSV and start creating folders

    studentRecords = [] # list of student name, id number, etc, and dict of google information
    studenetFailures = [] # list of students that had one failure or another
    gdClassOfFoldersDict = {} # dict of created or discovered Class Of xxxx folders
    
    ### check for BASE FOLDER before continuing
    # if it does not exist inform the user and bail
    logging.info('checking for student portfolio folder on google drive')
    try:
        gdBaseFolder = myDrive.getprops(fileId = BASEFOLDER, fields = 'webViewLink')
    except GDriveError as e:
        logging.critical('Error checking for student portfolio directory')
        logging.critical(e)
        return(False)
    except Exception as e:
        logging.critical('unexpected error: {}'.format(e))
        return(False)

    if not gdBaseFolder.get('webViewLink'):
        logging.critical('Fatal error: student portfolio directory not found')
        logging.critical('the supplied google drive URL is not valid; please update and try again')
        logging.critical(gdBaseFolder)
        return(False)

    ### loop through students
    logging.info('Searching for existing google drive folders and creating missing folders...')
    for index, student in enumerate(studentCSV):
    
        ## check if the Class OF folder exists, record in  (gdClassOfFoldersList)
        # it it does not exist, create it and record 
        # if more than one exists, inform the user

        classOf = 'Class Of-' + student[headerMap['ClassOf']]

        logging.info('searching google drive portfolio folder for folder: {}'.format(classOf))
        # check to see if 'Class Of' folder has been accessed before
        if classOf in gdClassOfFoldersDict:
            gdClassOfFolder = gdClassOfFoldersDict[classOf]
            logging.debug('folder has been previously accessed')
        else: # search google drive for the folder
            try:
                folder = myDrive.search(name = classOf, orderBy = 'createdTime', trashed = False, mimeType = 'folder')
            except GDriveError as e:
                logging.error('error processing student: {}'.format(student[headerMap['LastFirst']]))
                studentFailure.append(student)
                continue
            except Exception as e:
                logging.critical('Fatal error. Exiting')
                logging.critical(e)
                return(False)

            # check for results
            # always check for the oldest - [files][0] (set by orderBy = 'createdTime)
            if len(folder['files']) > 0:
                logging.debug('{:>5}{} folder found'.format('', classOf))
                gdClassOfFolder = folder['files'][0].get('id')
            else:
                logging.debug('{:>5}folder not found. creating.',format(''))
                # create a folder if no results were found
                try:
                    folder = myDrive.add(name = classOf, parents = BASEFOLDER, mimeType = 'folder')
                except GDriveError as e:
                    logging.error('error processing student {}'.format(student[headerMap['LastFirst']]))
                    studentFailure.append(student)
                    continue
                except Exception as e:
                    logging.critical('Fatal error. Exiting')
                    logging.critical(e)
                    return(False)

                gdClassOfFolder = folder

            gdClassOfFoldersDict[classOf] = gdClassOfFolder
            
        # end Class Of Folder check

        ## check if the student folder exists in the class of folder gdStudentFolder
        # if it does, record 
        # if it does not, create and record
        # if more than one exists, inform the user

        studentFolder = student[headerMap['LastFirst']] + ' - ' + student[headerMap['Student_Number']]

        logging.info('searching for student folder: {}'.format(studentFolder))

        try:
            folder = myDrive.search(name = studentFolder, trashed = False, mimeType = 'folder',
                                   parents = gdClassOfFoldersDict[classOf], orderBy = 'createdTime')
        except GDriveError as e:
            logging.error('error processing student {}'.format(student[headerMap['LastFirst']]))
            studentFailure.append(student)
            continue
        except Exception as e:
            logging.critical('Fatal error. Exiting')
            logging.critical(e)
            return(False)
        
        if len(folder['files']) > 0:
#             studentLinks.append(folder['files'][0]) # record this for generating CSV later            
            logging.debug('{:>5}getting properties of folder'.format(''))
            try:
                gdStudentFolder = myDrive.getprops(fileId = folder['files'][0].get('id'), fields = 'id, webViewLink' )
            except GDriveError as e:
                logging.error('error processing student {}'.format(student[headerMap['LastFirst']]))
                studentFailire.append(student)
                continue
            except Exception as e:
                logging.critical('Fatal error. Exiting')
                logging.critical(e)
                return(False)                
            
            logging.debug('{:>5}existing student folder found'.format(''))
        else:
            logging.debug('{:>5}creating student folder'.format(''))
            try:
                folder = myDrive.add(name = studentFolder, mimeType = 'folder', 
                                     parents = gdClassOfFoldersDict[classOf], fields = 'webViewLink, id')
                
            except GDriveError as e:
                logging.error('error processing student {}'.format(student[headerMap['FirstLast']]))
                studentFailure.append(student)
                continue
            except Exception as e:
                logging.critical('Fatal error. Exiting')
                logging.critical(e)
                return(False)
            logging.debug('folder created')
            gdStudentFolder = folder # save the gdStudentFolder for the grade folder check
            
        logging.debug('{:>5}student data: {}'.format('', student))
        logging.debug('{:>5}webViewLink: {}'.format('', folder.get('webViewLink')))
        student.append(gdStudentFolder.get('webViewLink'))
        studentLinks.append(student)
            
        
        # end of student folder
    
        ## loop through all of the grade folders (02-Grade, 03-Grade...) check if gdGradeFolder exists
        # if it does continue to next folder
        # if it does not, record
        for eachFolder in gradeFolders:
            
            logging.info('searching for grade folder: {}'.format(eachFolder))
            try:
                folder = myDrive.search(name = eachFolder, mimeType = 'folder', trashed = False, 
                                             parents = gdStudentFolder.get('id'), orderBy = 'createdTime')
            except GDriveError as e:
                logging.error('error processing student {}'.format(studnet[headerMap['FirstLast']]))
                studentFailure.append(student)
                continue
            except Exception as e:
                logging.critical('Fatal error. Exiting')
                logging.critical(e)
                return(False)
                
            if len(folder['files']) > 0:
                logging.debug('{:>5}folder exists, no action needed: {}'.format('', eachFolder))
                continue
            else:
                logging.debug('{:>5}creating folder: {}'.format('', eachFolder))
                try:
                    folder = myDrive.add(name = eachFolder, mimeType = 'folder', 
                                         parents = gdStudentFolder.get('id'))
                except GDriveError as e:
                    logging.error('error processing student {}')
                    studentFailure.append(student)
                    continue
                except Exception as e:
                    logging.critical('Fatal error. Exiting')
                    logging.critical(e)
                    return(False)
    
    ### Generate CSV output from Student Links
    csvHeader.append('StudentURL')
    headerFmtString = '{}, {}, {}\n'.format(csvHeader[headerMap['LastFirst']], csvHeader[headerMap['Student_Number']], 
                    csvHeader[-1])
    csvFmtString = '"{}", {}, "<a href={}>Student Portfolio for {}</a>"\n'
    studentURLData = os.path.expanduser('~/Desktop/Student_URL_Data.csv')

    logging.info('writing CSV file: {}'.format(studentURLData))
    try:
        with open(studentURLData, 'w') as f:
            f.write(headerFmtString)
            for eachRecord in studentLinks:
                f.write(csvFmtString.format(eachRecord[headerMap['LastFirst']], eachRecord[headerMap['Student_Number']], 
                        eachRecord[-1], eachRecord[headerMap['LastFirst']]))
    except (IOError, OSError) as e:
        print e

#     print(formatString.format(csvHeader[headerMap['LastFirst']], csvHeader[headerMap['Student_Number']], 
#                              csvHeader[-1]))
#     for eachRecord in studentLinks:
#         print(formatString.format(eachRecord[headerMap['LastFirst']], eachRecord[headerMap['Student_Number']], 
#                                  eachRecord[-1]))
    
    
    
        


In [10]:
foo = main()


[INFO     - root] checking google credentials
[INFO     - root] reading list of folders to create for each student:
[INFO     - root] Folders to be created for each student: 5
[INFO     - root]      List of folders to be created for each student:
[INFO     - root]           08-Grade
[INFO     - root]           09-Grade
[INFO     - root]           10-Grade
[INFO     - root]           11-Grade
[INFO     - root]           12-Grade
[INFO     - root] reading student CSV file
[INFO     - root] 12 student records found in ./student_export.text
[INFO     - root] checking for student portfolio folder on google drive
[DEBUG    - root] files().get(fileId=0B9WTleJ1MzaYcmdmTWNNNF9pa1E, fields=webViewLink)
[INFO     - root] Searching for existing google drive folders and creating missing folders...
[INFO     - root] searching google drive portfolio folder for folder: Class Of-2030
[DEBUG    - root] apicall: files().list(q=name = "Class Of-2030" and trashed=False and mimeType="application/vnd.google-

KeyboardInterrupt: 

In [ ]:
credentials = getCredentials()
http = credentials.authorize(httplib2.Http())
service = discovery.build('drive', 'v3', http=http)


testDrive = gDrive(service)


In [ ]:
testDrive.search(name='schmoo')

In [ ]:
foo = testDrive.getprops(fileId = '0B9WTleJ1MzaYcmdmTWNNNF9pa1E', fields = 'webViewLink')
print foo
if not foo.get('webViewLink'):
    print 'not there'
else:
    print 'got it'

In [ ]:
myDict = {'a':'eels', 'b':'hovercraft'}
if 'c' in myDict:
    print myDict['b']
else:
    print 'nope'

In [ ]:
foobar = testDrive.add(name = 'schmoo', parents = '0B9WTleJ1MzaYcmdmTWNNNF9pa1E', mimeType = 'folder', fields = 'id, parents, webViewLink, createdTime')
print foobar

In [ ]:
help(testDrive.add)